# Tarea 2

La meta de esta tarea es mejorar el 0,48 de accuracy que obtuvo el árbol de decisión entrenado para el dataset de helados, con un train/test split de 75-25. Partamos por cargar el dataset y el split. 

### Detalles administrativos

La tarea es individual. Para la entrega, sube este notebook al cuestionario del siding, el plazo para hacerlo es el **Viernes 24 de Septiembre, a las 20:00 hrs**. 

In [87]:
import pandas as pd
helados = pd.read_csv('Ice_cream.csv')
helados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Unnamed: 0  200 non-null    int64
 1   id          200 non-null    int64
 2   female      200 non-null    int64
 3   ice_cream   200 non-null    int64
 4   video       200 non-null    int64
 5   puzzle      200 non-null    int64
dtypes: int64(6)
memory usage: 9.5 KB


In [88]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(helados[["female","puzzle","video"]], helados["ice_cream"], test_size=0.25, random_state=42)


## Parte I: boosting simple

Lo primero que vas a hacer es un modelo de boosting simple. Implementa una clase que tenga metodos _fit_ y _predict_.

El método fit recibe un set X, y para entrenar, donde X es un dataframe e y es una serie de números 0 o 1, y hace lo siguiente: 
- primero entrena un árbol de decisión A1(por ejemplo, usando sklearn como vimos en clases) con X e y
- Sea C el conjunto de tuplas en X para los cuales A1 clasificó mal (es decir, el resultado de A1 para esa tupla es distinto de lo que tiene y), y sea y_c el pedazo de y correspondiente a esas tuplas. 
- ahora entrena un árbol de decisión A2, pero usando solo C, y_c. 

El método predict recibe un set X (del mismo número de dimensiones que el que se uso para fit), y ejecuta lo siguiente: 
- usa _predict_proba_ o un método similar para conseguir las probabilidades según el árbol A1 de que cada tupla en X sea clasificada como **1**
- usa _predict_proba_ o un método similar para conseguir las probabilidades según el árbol A2 de que cada tupla en X sea clasificada como **1**
- sea p la probabilidad calculada como 0,7 veces la proabilidad según A1 y 0,3 veces la probabilidad según A2. Tu clase predice clase 1 si p es **mayor** que 0.5, y si no predice un 0. 


In [89]:
from sklearn import tree
### escribe tu clase aquí
class BoostingSimple:
    def __init__(self):
        self.A1 = tree.DecisionTreeClassifier()
        self.A2 = tree.DecisionTreeClassifier()
    def fit(self, X,y):
        self.A1.fit(X,y)
        predA1 = self.A1.predict(X)
        Cbool = predA1 != y
        C = X[Cbool]
        c_y = y[Cbool]
        self.A2.fit(C,c_y)
    def predict(self,X):
        valA1 = self.A1.predict_proba(X)[:,1]
        valA2 = self.A2.predict_proba(X)[:,1]
        p = valA1 * 0.7 + valA2 * 0.3
        Rbool = p > 0.5
        return Rbool.astype(int)
        

Lo segundo: Usa ahora el set de datos de test para ver el accuracy score de tu clase cuando se entrena con los datos de train de los helados pero se testea con el set de test. ¿Cómo se compara con la del árbol 1? ¿Con la del árbol 2? 

In [90]:
from sklearn.metrics import accuracy_score
BoostTree = BoostingSimple()
BoostTree.fit(X_train,y_train)

Xboost = BoostTree.predict(X_test)
xA1 = BoostTree.A1.predict(X_test)
xA2 = BoostTree.A2.predict(X_test) 

print(f"Valor accuracy con boosting: {accuracy_score(Xboost,y_test)}") 
print(f"Valor accuracy de A1: {accuracy_score(xA1,y_test)}")
print(f"Valor accuracy de A2: {accuracy_score(xA2,y_test)}")



Valor accuracy con boosting: 0.54
Valor accuracy de A1: 0.48
Valor accuracy de A2: 0.5


#### Respuesta
Como se puede ver, el valor de accuracy realizado con boosting aumento a 0.54 aprox (aproximado ya que el arbol de decision tambien tiene variables aleatorias que podrian cambiar el accuracy score), en comparacion a 0.48  y 0.5 de A1 y A2, respectivamente.


## Parte 2: agregando árboles

Modifica acá abajo tu clase para que agregue un tercér árbol, que se entrene sobre el subconjunto de tuplas de C que el árbol 2 no logra clasificar bien, o bien (si este subconjunto es vacío), sobre el conjunto C' de tuplas de X que el clasificador de la parte I no logra clasificar bien. 

Busca si hay algún valor de pesos que puedes entregarles a los árboles en el método _predict_ para que tu nuevo clasificador mejore su accuracy score con el set de test (con pesos nos referimos a la ponderación que se le da a la probabilidad de cada árbol - para la clase con dos árboles los pesos fueron 0.7 y 0.3). Puedes limitar tu búsqueda a valores de alfa con un decimal: 0, 0.1, 0.2, hasta 0.9, 1. 


In [91]:
### escribe la clase modificada aquí
class BoostingSimple2:
    def __init__(self):
        self.A1 = tree.DecisionTreeClassifier()
        self.A2 = tree.DecisionTreeClassifier()
        self.A3 = tree.DecisionTreeClassifier()
    def fit(self, X,y):
        self.A1.fit(X,y)
        predA1 = self.A1.predict(X)
        Cbool = predA1 != y
        C = X[Cbool]
        c_y = y[Cbool]
        self.A2.fit(C,c_y)
        predA2 = self.A2.predict(C)
        Ccbool = predA2 != c_y
        Cc = C[Ccbool]
        cc_y = y[Cbool]
        self.A3.fit(Cc,cc_y)
    def predict(self,X):
        valA1 = self.A1.predict_proba(X)[:,1]
        valA2 = self.A2.predict_proba(X)[:,1]
        valA3 = self.A3.predict_proba(X)[:,1]
        p = valA1 * 0.6 + valA2 * 0.3 + valA3 * 0.1
        Rbool = p > 0.5
        return Rbool.astype(int)
        
class BoostingSimple3:
    def __init__(self):
        self.A1 = tree.DecisionTreeClassifier()
        self.A2 = tree.DecisionTreeClassifier()
        self.A3 = tree.DecisionTreeClassifier()
        self.boost = BoostingSimple()
    def fit(self, X,y):
        self.boost.fit(X,y)
        self.A1 = self.boost.A1
        self.A2 = self.boost.A2
        predboost = self.boost.predict(X)
        Cbool = predboost != y
        C = X[Cbool]
        c_y = y[Cbool]
        self.A3.fit(C,c_y)
        
    def predict(self,X):
        valA1 = self.A1.predict_proba(X)[:,1]
        valA2 = self.A2.predict_proba(X)[:,1]
        valA3 = self.A3.predict_proba(X)[:,1]
        p = valA1 * 0.4 + valA2 * 0.1 + valA3 * 0.5
        Rbool = p > 0.5
        return Rbool.astype(int)







# Aviso
Como no se encontraban tuplas evaluadas incorrectamente por A2, se definio otra clase boost que entrena A3 a partir de los valores incorrectos de la clase anterior.

In [92]:
### busca el alfa o reporta el alfa encontrado acá. 
BoostTree3 = BoostingSimple3()
BoostTree3.fit(X_train,y_train)
Yboost = BoostTree3.predict(X_test)
yA1 = BoostTree3.A1.predict(X_test)
yA2 = BoostTree3.A2.predict(X_test) 
yA3 = BoostTree3.A2.predict(X_test) 
yboost = BoostTree3.boost.predict(X_test)

print(f"Valor accuracy con boosting nuevo: {accuracy_score(Yboost,y_test)}") 
print(f"Valor accuracy con boosting antiguo: {accuracy_score(yboost,y_test)}") 
print(f"Valor accuracy de A1: {accuracy_score(yA1,y_test)}")
print(f"Valor accuracy de A2: {accuracy_score(yA2,y_test)}")
print(f"Valor accuracy de A3: {accuracy_score(yA3,y_test)}")
print(f"Accuracy score entre A2-A3: {accuracy_score(yA3,yA2)}")


Valor accuracy con boosting nuevo: 0.6
Valor accuracy con boosting antiguo: 0.54
Valor accuracy de A1: 0.48
Valor accuracy de A2: 0.5
Valor accuracy de A3: 0.5
Accuracy score entre A2-A3: 1.0
